# Importación de librerías

In [15]:
import pandas as pd
import numpy as np

import os
import librosa
import librosa.display

import matplotlib.pyplot as plt
import seaborn as sns


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Lectura del dataset

In [17]:
urbansound8k_path = "../../data/raw/UrbanSound8K/"

# Primer dataset - Carpeta con los audios
urbansound8k_audio_path = "audio/"

# Segundo dataset - Archivo CSV con metadatos sobre los audios
urbansound8k_metadata_path_file = "metadata/UrbanSound8K.csv"

# Dataset CSV con metadatos consolidados
dataset_urbansound8k_vfinal_path_file = "../../data/final/dataset_urbansound8k_vfinal.csv"

# Variable "clase"
variable_predict =  'classID'

In [18]:
# Lectura
dataset_urbansound8k_df = pd.read_csv(dataset_urbansound8k_vfinal_path_file, sep=";")

dataset_urbansound8k_df.head(3)

,classID,tasa_cruce_cero,entropia_espectograma,avg_centroide_espectral,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13
0,3,1947,6.716659,2503.702337,-275.91843,119.49280,-98.21178,-66.515140,-42.606050,0.505066,-28.330938,-5.746867,9.992785,4.795411,15.461894,-0.069880,-2.842674
1,2,27607,9.991457,2339.221804,-500.90840,185.10641,-86.53282,49.858850,9.230822,22.548956,-3.567175,12.220052,7.720082,-6.460391,16.995657,-6.625116,1.469779
2,2,20971,9.728703,2070.826863,-531.19530,186.93994,-70.34916,40.429245,9.121047,18.398588,6.283282,15.504061,9.613501,-7.113610,16.179823,-5.710522,-0.899251


In [19]:
# Columnas disponibles del dataset
dataset_urbansound8k_df.columns

Index(['classID', 'tasa_cruce_cero', 'entropia_espectograma',
       'avg_centroide_espectral', 'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4',
       'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11',
       'mfcc_12', 'mfcc_13'],
      dtype='object')

# Generación datos de entrenamiento y prueba

In [20]:
# Separar las características y la variable objetivo
# ['tasa_cruce_cero', 'entropia_espectograma', 'avg_centroide_espectral'
columnas_caracteristicas = ['mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7',
             'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13']

X = dataset_urbansound8k_df[columnas_caracteristicas]  # Características
y = dataset_urbansound8k_df[variable_predict]  # Variable "clase" o "objetivo"

In [21]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Generar un dataset a partir de los datos de entrenamiento, para busqueda de hiperparametros 
dataset_train_df = pd.DataFrame(X_train) 
dataset_train_df[variable_predict] = y_train

dataset_train_df.reset_index(drop=True, inplace=True)
dataset_train_df.head(3)

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,classID
0,-159.67445,181.50089,-92.606600,-15.836291,-32.712960,-0.886196,-2.333837,21.515417,25.863836,-0.396893,-0.778924,9.013906,6.754499,8
1,-219.52864,177.02522,-32.444240,26.932178,-21.066864,4.041955,-10.886847,-7.922688,-13.082119,-10.993799,-3.332730,-5.833699,-5.191674,9
2,-320.74875,205.43745,-47.278393,-87.469260,-26.390732,-40.026196,-48.005367,16.560823,49.147522,14.839294,-4.111610,4.170245,1.705174,8


# Funcion utilitarias

In [22]:
def print_linea_separadora(n=20):
    print("-"*20)    

def mostrar_resultados_randomizedSearch(random_search):
    # Obtener los mejores parámetros y el mejor score
    best_params = random_search.best_params_
    best_score = random_search.best_score_

    print(f"Mejores Parámetros: {best_params}")
    print(f"Mejor Score: {best_score}")
    print_linea_separadora()    

    pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
    pd.set_option('display.max_colwidth', None) 

    # Obtener el DataFrame con los resultados
    results = pd.DataFrame(random_search.cv_results_)

    # Ordenar los resultados por la métrica de puntuación 
    top_results = results.sort_values(by='mean_test_score', ascending=False).head(5)
    display(top_results[['params', 'mean_test_score', 'rank_test_score']])

def evaluar_modelo_con_mejor_estimador(random_search, X_test, y_test):
    # Evaluar el modelo con los mejores parámetros en el conjunto de prueba
    best_knn = random_search.best_estimator_
    y_pred = best_knn.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy en conjunto de prueba: {accuracy}")
    print_linea_separadora()

# Modelos de Clasificacion

## KNeighborsClassifier

### Randomized

In [23]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Aquí podrías cargar tus datos
# X_train, X_test, y_train, y_test = ...

# Definir el modelo KNN
knn = KNeighborsClassifier()

# Definir la rejilla de parámetros para RandomizedSearchCV
knn_param_distributions = {
    'n_neighbors': np.arange(1, 30),            # Número de vecinos (de 1 a 29)
    'weights': ['uniform', 'distance'],         # Tipo de ponderación
    'metric': ['euclidean', 'manhattan', 'minkowski']  # Métricas de distancia
}

# Configurar RandomizedSearchCV
knn_randomized_search = RandomizedSearchCV(
    estimator=knn, 
    param_distributions=knn_param_distributions, 
    n_iter=10,  # Número de combinaciones aleatorias a probar
    cv=10,  # Validación cruzada con 10 pliegues
    scoring='accuracy',  # Métrica para optimizar
    n_jobs=-1,  # Usar todos los núcleos disponibles
    random_state=42  # Fijar la semilla para reproducibilidad
)

# Entrenar el modelo con la búsqueda de hiperparámetros
knn_randomized_search.fit(X_train, y_train)


RandomizedSearchCV(cv=10, estimator=KNeighborsClassifier(), n_jobs=-1,
                   param_distributions={'metric': ['euclidean', 'manhattan',
                                                   'minkowski'],
                                        'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
                                        'weights': ['uniform', 'distance']},
                   random_state=42, scoring='accuracy')

In [24]:
mostrar_resultados_randomizedSearch(knn_randomized_search)

evaluar_modelo_con_mejor_estimador(knn_randomized_search, X_test, y_test)

Mejores Parámetros: {'weights': 'distance', 'n_neighbors': 4, 'metric': 'manhattan'}
Mejor Score: 0.8830334780345233
--------------------


,params,mean_test_score,rank_test_score
9,"{'weights': 'distance', 'n_neighbors': 4, 'metric': 'manhattan'}",0.883033,1
3,"{'weights': 'distance', 'n_neighbors': 6, 'metric': 'minkowski'}",0.841945,2
7,"{'weights': 'distance', 'n_neighbors': 8, 'metric': 'euclidean'}",0.827914,3
8,"{'weights': 'uniform', 'n_neighbors': 6, 'metric': 'minkowski'}",0.797278,4
2,"{'weights': 'distance', 'n_neighbors': 22, 'metric': 'manhattan'}",0.796275,5


Accuracy en conjunto de prueba: 0.8986834573554665
--------------------


## GMM

### RandomizedSearchCV

In [25]:
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Definir el modelo GMM
gmm = GaussianMixture()

# Definir el espacio de búsqueda de hiperparámetros
gmm_param_distributions = {
    'n_components': np.arange(1, 21),  # Número de componentes en la mezcla
    'covariance_type': ['full', 'tied', 'diag', 'spherical'],  # Tipo de covarianza
    'tol': np.logspace(-4, -1, 10),    # Tolerancia de convergencia
    'reg_covar': np.logspace(-6, -1, 10),  # Regularización para la covarianza
}

# Configurar RandomizedSearchCV
gmm_randomized_search = RandomizedSearchCV(
    estimator=gmm,
    param_distributions=gmm_param_distributions,
    n_iter=20,                          # Número de combinaciones a probar
    cv=10,                              # Número de pliegues para la validación cruzada
    verbose=2,                          # Verbosidad del proceso
    random_state=42,                    # Semilla para reproducibilidad
    n_jobs=-1                           # Usar todos los núcleos disponibles
)

# Ajustar RandomizedSearchCV
gmm_randomized_search.fit(X_train)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


c:\Users\colbe\Documents\VIU\TFM\tfm-clasificador-fuentes-sonoras-ciudad\.env\lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
  warnings.warn(


RandomizedSearchCV(cv=10, estimator=GaussianMixture(), n_iter=20, n_jobs=-1,
                   param_distributions={'covariance_type': ['full', 'tied',
                                                            'diag',
                                                            'spherical'],
                                        'n_components': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20]),
                                        'reg_covar': array([1.00000000e-06, 3.59381366e-06, 1.29154967e-05, 4.64158883e-05,
       1.66810054e-04, 5.99484250e-04, 2.15443469e-03, 7.74263683e-03,
       2.78255940e-02, 1.00000000e-01]),
                                        'tol': array([0.0001    , 0.00021544, 0.00046416, 0.001     , 0.00215443,
       0.00464159, 0.01      , 0.02154435, 0.04641589, 0.1       ])},
                   random_state=42, scoring='accuracy', verbose=2)

In [26]:
mostrar_resultados_randomizedSearch(gmm_randomized_search)

evaluar_modelo_con_mejor_estimador(gmm_randomized_search, X_test, y_test)

Mejores Parámetros: {'tol': 0.0001, 'reg_covar': 0.007742636826811277, 'n_components': 13, 'covariance_type': 'spherical'}
Mejor Score: nan
--------------------


,params,mean_test_score,rank_test_score
0,"{'tol': 0.0001, 'reg_covar': 0.007742636826811277, 'n_components': 13, 'covariance_type': 'spherical'}",NaN,1
1,"{'tol': 0.001, 'reg_covar': 1e-06, 'n_components': 17, 'covariance_type': 'spherical'}",NaN,1
2,"{'tol': 0.0001, 'reg_covar': 0.0021544346900318843, 'n_components': 9, 'covariance_type': 'full'}",NaN,1
3,"{'tol': 0.0001, 'reg_covar': 0.1, 'n_components': 14, 'covariance_type': 'diag'}",NaN,1
4,"{'tol': 0.01, 'reg_covar': 1.2915496650148827e-05, 'n_components': 13, 'covariance_type': 'diag'}",NaN,1


Accuracy en conjunto de prueba: 0.10303377218088151
--------------------


## Random Forest

### Randomized

In [27]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Definir el modelo Random Forest
rf = RandomForestClassifier()

# Definir el espacio de búsqueda de hiperparámetros
rf_param_distributions = {
    'n_estimators': np.arange(10, 200, 10),         # Número de árboles en el bosque
    'max_features': ['auto', 'sqrt', 'log2'],       # Número de características a considerar en cada división
    'max_depth': np.arange(10, 110, 10).tolist() + [None],  # Máxima profundidad del árbol
    'min_samples_split': np.arange(2, 11),          # Número mínimo de muestras necesarias para dividir un nodo
    'min_samples_leaf': np.arange(1, 11),           # Número mínimo de muestras en una hoja
    'bootstrap': [True, False]                      # Método de muestreo para los árboles
}

# Configurar RandomizedSearchCV
rf_randomized_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=rf_param_distributions,
    n_iter=100,                        # Número de combinaciones a probar
    cv=10,                              # Número de pliegues para la validación cruzada
    verbose=2,                          # Verbosidad del proceso
    scoring='accuracy',  # Métrica para optimizar
    random_state=42,                    # Semilla para reproducibilidad
    n_jobs=-1                           # Usar todos los núcleos disponibles
)

# Ajustar RandomizedSearchCV
rf_randomized_search.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


c:\Users\colbe\Documents\VIU\TFM\tfm-clasificador-fuentes-sonoras-ciudad\.env\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
400 fits failed out of a total of 1000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
212 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\colbe\Documents\VIU\TFM\tfm-clasificador-fuentes-sonoras-ciudad\.env\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\colbe\Documents\VIU\TFM\tfm-clasificador-fuentes-sonoras-ciudad\.env\lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\colbe\

RandomizedSearchCV(cv=10, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
       140, 150, 160, 170, 180, 190])},
                   random_state=42, scoring='accuracy', verbose=2)

In [28]:
mostrar_resultados_randomizedSearch(rf_randomized_search)

evaluar_modelo_con_mejor_estimador(rf_randomized_search, X_test, y_test)

Mejores Parámetros: {'n_estimators': 60, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 90, 'bootstrap': False}
Mejor Score: 0.8781650823321078
--------------------


,params,mean_test_score,rank_test_score
85,"{'n_estimators': 60, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 90, 'bootstrap': False}",0.878165,1
24,"{'n_estimators': 160, 'min_samples_split': 5, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'max_depth': 100, 'bootstrap': False}",0.875730,2
75,"{'n_estimators': 90, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': False}",0.875301,3
64,"{'n_estimators': 170, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_features': 'log2', 'max_depth': 100, 'bootstrap': False}",0.865707,4
52,"{'n_estimators': 150, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 20, 'bootstrap': True}",0.865277,5


Accuracy en conjunto de prueba: 0.8958214081282198
--------------------


## SVM

### Randomized

In [15]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Definir el clasificador SVM
svm = SVC()

# Definir el espacio de búsqueda de hiperparámetros
svm_param_distributions = {
    'C': np.logspace(-3, 3, 7),            # Parámetro de penalización
    'gamma': np.logspace(-3, 3, 7),        # Parámetro del núcleo RBF
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']  # Tipos de núcleos
}

# Configurar RandomizedSearchCV
svm_randomized_search = RandomizedSearchCV(
    estimator=svm,
    param_distributions=svm_param_distributions,
    n_iter=20,                          # Número de combinaciones a probar
    cv=5,                              # Número de pliegues para la validación cruzada
    verbose=2,                          # Verbosidad del proceso
    random_state=42,                    # Semilla para reproducibilidad
    n_jobs=-1                           # Usar todos los núcleos disponibles
)

# Ajustar RandomizedSearchCV
svm_randomized_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


: 

In [ ]:
mostrar_resultados_randomizedSearch(svm_randomized_search)
evaluar_modelo_con_mejor_estimador(svm_randomized_search, X_test, y_test)